In [1]:
%%bash
curl -s http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip --output ~/Downloads/baseballdata.zip

In [2]:
%%bash
mkdir ~/Downloads/baseballdata/
unzip -qq ~/Downloads/baseballdata.zip -d ~/Downloads/baseballdata/

In [3]:
import os

home = os.path.join(os.path.expanduser('~'), 'Downloads/')

In [4]:
import sqlalchemy
import pandas as pd

con = sqlalchemy.create_engine('sqlite:///{}baseball.sqlite'.format(home))

This is a bit unconventional, but below we load each CSV to a pandas DataFrame, then dump all into a single SQLite database.  This better mimics the way we might actually use `sqlalchemy`. (Interacting with databases, not flat files.)

One thing we need to be aware of with this process is that we might lose data types.

In [5]:
import glob

for filepath in glob.glob('{}baseballdata/*.csv'.format(home)):
    table = pd.read_csv(filepath)
    name = os.path.basename(filepath).split('.')[0]
    table.to_sql(name=name, con=con)

# Question 1

<font color="blue">
What was the total spent on salaries by each team, each year?
</font>

Here are these records for the first 5 teams in 1985.

But first, let's define a helper for dumping queries to a DataFrame.

Again, this is a bit redundant knowing that we could use `pd.read_sql_query` here, so is for illustration only:

In [6]:
def sql_to_df(con, stmt, **kwargs):
    res = con.execute(stmt).fetchall()
    return pd.DataFrame(res, **kwargs)

In [7]:
stmt = """
SELECT yearID, teamID, SUM(salary)
FROM Salaries
GROUP BY yearID, teamID;
"""

In [8]:
print(sql_to_df(con, stmt,
                columns=['year', 'team', 'total_salary']).head())

   year team  total_salary
0  1985  ATL      14807000
1  1985  BAL      11560712
2  1985  BOS      10897560
3  1985  CAL      14427894
4  1985  CHA       9846178


# Question 2

<font color="blue">
What is the first and last year played for each player?
</font>

In [9]:
stmt = """
SELECT PlayerID, MIN(yearID), MAX(yearID) FROM Fielding GROUP BY playerID;
"""

In [10]:
print(sql_to_df(con, stmt,
                columns=['playerid', 'min_yr', 'max_yr']).head())

    playerid  min_yr  max_yr
0  aardsda01    2004    2013
1  aaronha01    1954    1976
2  aaronto01    1962    1971
3   aasedo01    1977    1990
4   abadan01    2001    2003


# Question 3

<font color="blue">
Who has played the most all star games?
</font>

In [11]:
stmt = """
SELECT nameFirst, nameLast
FROM Master
WHERE playerID = (
    SELECT playerID
    FROM AllstarFull
    GROUP BY playerID
    ORDER BY COUNT(playerID) DESC
    LIMIT 1);
"""

In [12]:
' '.join(con.execute(stmt).fetchone())

'Hank Aaron'

# Question 4

<font color="blue">
Which school has generated the most distinct players?
</font>

In [13]:
stmt = """
SELECT schoolName
FROM Schools
WHERE schoolID = (
    SELECT schoolID
    FROM SchoolsPlayers
    GROUP BY schoolID
    ORDER BY COUNT(playerID) DESC
    LIMIT 1);
"""

In [14]:
con.execute(stmt).fetchone()

('University of Southern California',)

# Question 5

<font color="blue">
Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.
</font>

In [15]:
stmt = """
SELECT nameFirst, nameLast
FROM Master
ORDER BY DATE(finalGame) - DATE(debut) DESC
LIMIT 1;
"""

In [16]:
' '.join(con.execute(stmt).fetchone())

'Nick Altrock'

From Wikipedia:

> Nicholas "Nick" Altrock (September 15, 1876 – January 20, 1965) was an American left-handed pitcher in Major League Baseball. Though his days as a full-time player ended quickly due to injury, Altrock made periodic appearances as a pinch hitter for many years. He appeared in a game at the age of 57.

# Question 6

<font color="blue">
What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.
</font>

No `EXTRACT` in SQLite; the equivalent would be `STRFTIME`.

In [17]:
stmt = """
SELECT STRFTIME('%m', DATE(debut)) AS month, COUNT(STRFTIME('%m', DATE(debut))) 
FROM Master
WHERE month NOT NULL
GROUP BY STRFTIME('%m', DATE(debut));
"""

In [18]:
print(sql_to_df(con, stmt,
                columns=['month', 'counts']))

  month  counts
0    03      41
1    04    4711
2    05    2230
3    06    1893
4    07    1978
5    08    1943
6    09    5061
7    10     308


# Question 7

<font color="blue">
What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.
</font>

Sorry, it's not clear to me what this question is asking.  The results appear to be equal in this case.

In [19]:
stmt1 = """
SELECT nameFirst, nameLast, MeanSalaries.AvgSalary
FROM Master
JOIN (
    SELECT playerID, AVG(salary) as AvgSalary
    FROM Salaries
    GROUP BY playerID) MeanSalaries
ON Master.playerID = MeanSalaries.playerID;
"""

In [20]:
stmt2 = """
SELECT master.nameFirst, master.nameLast, MeanSalaries.AvgSalary
FROM (
    SELECT playerID, AVG(salary) as AvgSalary
    FROM Salaries
    GROUP BY playerID) MeanSalaries
JOIN Master
ON Master.playerID = MeanSalaries.playerID;
"""

In [21]:
df1 = sql_to_df(con, stmt1)

In [22]:
df2 = sql_to_df(con, stmt2)

In [23]:
from pandas.testing import assert_frame_equal

In [24]:
assert_frame_equal(df1, df2)